In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from pathlib import Path
import pandas as pd

In [9]:
# Create a file path
melb_data = Path('Resources/melbourne_listings.csv')
melb_full_df = pd.read_csv(melbourne_data)

# Read in the data.
melb_full_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1044679652057774327,https://www.airbnb.com/rooms/1044679652057774327,20240316023834,2024-03-18,previous scrape,Jubilee park home,Great location:3mins walking to the large aqua...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,378158616,...,NaN,NaN,NaN,NaN,f,2,2,0,0,NaN
1,38971182,https://www.airbnb.com/rooms/38971182,20240316023834,2024-03-18,city scrape,Furnished accomodation,close to Hawthorn station,NaN,https://a0.muscache.com/pictures/95071a26-902a...,298550539,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
2,1036489765594430755,https://www.airbnb.com/rooms/1036489765594430755,20240316023834,2024-03-18,city scrape,Lovely Ringwood Studio w/ Ensuite,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,551026724,...,5.00,5.00,5.00,NaN,t,10,0,10,0,1.27
3,28139568,https://www.airbnb.com/rooms/28139568,20240316023834,2024-03-16,previous scrape,Close to Monash Caulfield Short lease!!!,NaN,NaN,https://a0.muscache.com/pictures/48b8ed68-d51c...,212558156,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
4,572431386115618941,https://www.airbnb.com/rooms/572431386115618941,20240316023834,2024-03-16,city scrape,Selina St Kilda - Bed in 4-Bed Dormitory Room,You'll enjoy your time at this cheerful getaway.,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,446636199,...,4.43,4.37,3.97,NaN,t,8,0,4,4,1.21


In [11]:
melb_refined_df = melb_full_df.drop(columns=["listing_url", "scrape_id", "last_scraped", "source", "description", "neighborhood_overview", "picture_url", "host_url", "host_name", "host_about", "host_thumbnail_url", "host_picture_url", "neighbourhood_group_cleansed", "calendar_updated", "calendar_last_scraped", "license"])
melb_refined_df                     

,id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1044679652057774327,Jubilee park home,378158616,2020-12-03,"Box Hill South, Australia",within an hour,100%,79%,f,NaN,...,NaN,NaN,NaN,NaN,f,2,2,0,0,NaN
1,38971182,Furnished accomodation,298550539,2019-09-28,NaN,a few days or more,0%,0%,f,Hawthorn,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
2,1036489765594430755,Lovely Ringwood Studio w/ Ensuite,551026724,2023-12-14,NaN,NaN,NaN,NaN,NaN,NaN,...,5.00,5.00,5.00,5.00,t,10,0,10,0,1.27
3,28139568,Close to Monash Caulfield Short lease!!!,212558156,2018-08-30,"Melbourne, Australia",NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
4,572431386115618941,Selina St Kilda - Bed in 4-Bed Dormitory Room,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,NaN,...,4.60,4.43,4.37,3.97,t,8,0,4,4,1.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25082,925945938663212093,67floor Skyview 2BR 3beds for 6 centre of CBD,516470484,2023-05-25,"Melbourne, Australia",within an hour,100%,93%,t,NaN,...,4.59,4.96,4.98,4.84,f,5,5,0,0,6.09
25083,13865317,Brand New 3 Level Townhouse,81961829,2016-07-05,"Victoria, Australia",NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
25084,1038864675062787977,Luxe & Modern in heart of Sth Yarra- FREE Parking,72786492,2016-05-18,"Melbourne, Australia",within an hour,100%,100%,f,NaN,...,5.00,5.00,4.91,5.00,t,1,1,0,0,3.30
25085,14077119,Welcome to the 19th Century THE GEORGE' APART...,84598008,2016-07-18,"Saint Kilda, Australia",NaN,NaN,100%,f,St Kilda,...,4.97,4.97,4.90,4.94,t,1,1,0,0,0.71
